In [1]:
import os 
# kafka = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"
# spark_k = "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0"
kafka_spark = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2"
os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages {}  pyspark-shell".format(kafka_spark))

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
KAFKA_BROKER = "localhost:9092"
KAFKA_TOPIC = "sample"

sc = SparkContext('local')
spark = SparkSession(sc)
# spark = SparkSession.builder.master("local").appName("hi").getOrCreate()

In [4]:
spark

## Process data from kafka to dataframe by spark

In [53]:
df = spark.readStream.\
        format("kafka").\
        option("kafka.bootstrap.servers",KAFKA_BROKER).\
        option("subscribe",KAFKA_TOPIC).\
        load()


In [54]:
message = df.selectExpr("CAST(value as string)")

In [57]:
rawmessage = message.writeStream.queryName("test3").format("memory").start()

In [58]:
rawmessage.status

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

## Write data to hdfs

In [ ]:
fileStream = message.writeStream.\
                    queryName("Persist processed data 2").\
                    format("parquet").\
                    outputMode("append").\
                    option("path", "hdfs://localhost:9000/test/viethoan").\
                    option("checkpointLocation", "hdfs://localhost:9000/test/viethoan/checkpoint").\
                    start().\
                    awaitTermination()
